In [1]:
import torch
from pathlib import Path
import os
import numpy as np
import torch.nn as nn
from datetime import datetime

import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import argparse, os, shutil, time, warnings

from fp16util import *
from resnet import *
from PIL import Image
from torch.nn.parameter import Parameter

In [2]:

def get_parser():
    parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
    parser.add_argument('data', metavar='DIR', help='path to dataset')
    parser.add_argument('--save-dir', type=str, default=Path.cwd(), help='Directory to save logs and models.')
    parser.add_argument('-j', '--workers', default=8, type=int, metavar='N',
                        help='number of data loading workers (default: 4)')
    parser.add_argument('--momentum', default=0.9, type=float, metavar='M', help='momentum')
    parser.add_argument('--weight-decay', '--wd', default=5e-4, type=float,
                        metavar='W', help='weight decay (default: 1e-4)')
    parser.add_argument('-b', '--batch-size', default=256, type=int,
                        metavar='N', help='mini-batch size (default: 256)')
    parser.add_argument('--phases', default='[(0,2e-1,16),(2e-1,1e-2,16),(1e-2,0,5)]', type=str,
                    help='Should be a string formatted like this: [(start_lr,end_lr,num_epochs),(phase2...)]')
    parser.add_argument('--verbose', action='store_true', help='Verbose logging')
#     parser.add_argument('--init-bn0', action='store_true', help='Intialize running batch norm mean to 0')
    parser.add_argument('--print-freq', '-p', default=200, type=int,
                        metavar='N', help='print every this many steps (default: 5)')
#     parser.add_argument('--no-bn-wd', action='store_true', help='Remove batch norm from weight decay')
    parser.add_argument('--full-precision', action='store_true', help='Run model full precision mode. Default fp16')
    parser.add_argument('--loss-scale', type=float, default=512,
                        help='Loss scaling, positive power of 2 values can improve fp16 convergence.')
    parser.add_argument('--distributed', action='store_true', help='Run distributed training')
    parser.add_argument('--world-size', default=-1, type=int, 
                        help='total number of processes (machines*gpus)')
    parser.add_argument('--scale-lr', type=float, default=1, help='You should learning rate propotionally to world size')
    parser.add_argument('--dist-url', default='env://', type=str,
                        help='url used to set up distributed training')
    parser.add_argument('--dist-backend', default='nccl', type=str, help='distributed backend')
    parser.add_argument('--local_rank', default=0, type=int,
                        help='Used for multi-process training. Can either be manually set ' +
                        'or automatically set by using \'python -m multiproc\'.')
    return parser


In [3]:
args_input = [
    str(Path.home()/'data/cifar10/'),
    '--phases', '[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]',
#     '--phases', '[(0,2e-1,16),(2e-1,1e-2,16),(1e-2,0,5)]'
#     '--full-precision'
]

In [4]:
global args
args = get_parser().parse_args(args_input)
if args.full_precision: args.loss_scale = 1
torch.backends.cudnn.benchmark = True

## Model

In [5]:
# --
# Model definition
# Derived from models in `https://github.com/kuangliu/pytorch-cifar`
class PreActBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        self.bn1   = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        return out + shortcut


class ResNet18(nn.Module):
    def __init__(self, num_blocks=[2, 2, 2, 2], num_classes=10):
        super().__init__()
        
        self.in_channels = 64
        
        self.prep = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.layers = nn.Sequential(
            self._make_layer(64, 64, num_blocks[0], stride=1),
            self._make_layer(64, 128, num_blocks[1], stride=2),
            self._make_layer(128, 256, num_blocks[2], stride=2),
            self._make_layer(256, 256, num_blocks[3], stride=2),
        )
        
        self.classifier = nn.Linear(512, num_classes)
        
    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        
        strides = [stride] + [1] * (num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(PreActBlock(in_channels=in_channels, out_channels=out_channels, stride=stride))
            in_channels = out_channels
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.prep(x)
        
        x = self.layers(x)
        
        x_avg = F.adaptive_avg_pool2d(x, (1, 1))
        x_avg = x_avg.view(x_avg.size(0), -1)
        
        x_max = F.adaptive_max_pool2d(x, (1, 1))
        x_max = x_max.view(x_max.size(0), -1)
        
        x = torch.cat([x_avg, x_max], dim=-1)
        
        x = self.classifier(x)
        
        return x

### Torch loader

In [6]:
def pad(img, p=4, padding_mode='reflect'):
    return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))

def torch_loader(data_path, size, bs, val_bs=None):
    os.makedirs(data_path,exist_ok=True)

    val_bs = val_bs or bs
    # Data loading code
    tfms = [transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.24703,0.24349,0.26159))]
    train_tfms = transforms.Compose([
        pad, # TODO: use `padding` rather than assuming 4
        transforms.RandomCrop(size),
        transforms.RandomHorizontalFlip(),
    ] + tfms)
    val_tfms = transforms.Compose(tfms)

    train_dataset = datasets.CIFAR10(root=data_path, train=True, download=(args.local_rank==0), transform=train_tfms)
    val_dataset  = datasets.CIFAR10(root=data_path, train=False, download=(args.local_rank==0), transform=val_tfms)

    train_sampler = (torch.utils.data.distributed.DistributedSampler(train_dataset) if args.distributed else None)
    # val_sampler = (torch.utils.data.distributed.DistributedSampler(val_dataset) if args.distributed else None)
    val_sampler = None

    train_loader = DataLoader(
        train_dataset, batch_size=bs, shuffle=(train_sampler is None),
        num_workers=args.workers, pin_memory=True,
        sampler=train_sampler)

    val_loader = DataLoader(
        val_dataset, batch_size=val_bs, shuffle=False,
        num_workers=args.workers, pin_memory=True,
        sampler=val_sampler)
    
    train_loader = DataPrefetcher(train_loader)
    val_loader = DataPrefetcher(val_loader)
    
    return train_loader, val_loader

# Seems to speed up training by ~2%
class DataPrefetcher():
    def __init__(self, loader, stop_after=None):
        self.loader = loader
        self.dataset = loader.dataset
        self.stream = torch.cuda.Stream()
        self.stop_after = stop_after
        self.next_input = None
        self.next_target = None

    def __len__(self):
        return len(self.loader)

    def preload(self):
        try:
            self.next_input, self.next_target = next(self.loaditer)
        except StopIteration:
            self.next_input = None
            self.next_target = None
            return
        with torch.cuda.stream(self.stream):
            self.next_input = self.next_input.cuda(async=True)
            self.next_target = self.next_target.cuda(async=True)

    def __iter__(self):
        count = 0
        self.loaditer = iter(self.loader)
        self.preload()
        while self.next_input is not None:
            torch.cuda.current_stream().wait_stream(self.stream)
            input = self.next_input
            target = self.next_target
            self.preload()
            count += 1
            yield input, target
            if type(self.stop_after) is int and (count > self.stop_after):
                break

### Learning rate scheduler

In [7]:
class Scheduler():
    def __init__(self, optimizer, phases=[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]):
        self.optimizer = optimizer
        self.current_lr = None
        self.phases = phases
        self.tot_epochs = sum([p[2] for p in phases])

    def linear_lr(self, start_lr, end_lr, epoch_curr, batch_curr, epoch_tot, batch_tot):
        if args.scale_lr != 1:
            start_lr *= args.scale_lr
            end_lr *= args.scale_lr
        step_tot = epoch_tot * batch_tot
        step_curr = epoch_curr * batch_tot + batch_curr
        step_size = (end_lr - start_lr)/step_tot
        return start_lr + step_curr * step_size
    
    def get_current_phase(self, epoch):
        epoch_accum = 0
        for phase in self.phases:
            start_lr,end_lr,num_epochs = phase
            if epoch <= epoch_accum+num_epochs: return start_lr, end_lr, num_epochs, epoch - epoch_accum
            epoch_accum += num_epochs
        raise Exception('Epoch out of range')
            
    def get_lr(self, epoch, batch_curr, batch_tot):
        start_lr, end_lr, num_epochs, relative_epoch = self.get_current_phase(epoch)
        return self.linear_lr(start_lr, end_lr, relative_epoch, batch_curr, num_epochs, batch_tot)

    def update_lr(self, epoch, batch_num, batch_tot):
        lr = self.get_lr(epoch, batch_num, batch_tot)
        if args.verbose and (self.current_lr != lr) and ((batch_num == 1) or (batch_num == batch_tot)): 
            print(f'Changing LR from {self.current_lr} to {lr}')

        self.current_lr = lr

        for param_group in self.optimizer.param_groups:
            lr_old = param_group['lr'] or lr
            param_group['lr'] = lr

In [8]:
# item() is a recent addition, so this helps with backward compatibility.
def to_python_float(t):
    if isinstance(t, float): return t
    if isinstance(t, int): return t
    if hasattr(t, 'item'): return t.item()
    else: return t[0]

def train(trn_loader, model, criterion, optimizer, scheduler, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()
    end = time.time()

    st = time.time()
    trn_len = len(trn_loader)

    # print('Begin training loop:', st)
    for i,(input,target) in enumerate(trn_loader):
        batch_size = input.size(0)
        batch_num = i+1
        
        # measure data loading time
        scheduler.update_lr(epoch, i+1, trn_len)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        if args.distributed:
            # Must keep track of global batch size, since not all machines are guaranteed equal batches at the end of an epoch
            corr1 = correct(output.data, target)[0]
            metrics = torch.tensor([batch_size, loss, corr1]).float().cuda()
            batch_total, reduced_loss, corr1 = sum_tensor(metrics)
            reduced_loss = reduced_loss/dist.get_world_size()
            prec1 = corr1*(100.0/batch_total)
        else:
            reduced_loss = loss.data
            batch_total = input.size(0)
            prec1 = accuracy(output.data, target)[0] # measure accuracy and record loss
        losses.update(to_python_float(reduced_loss), to_python_float(batch_total))
        top1.update(to_python_float(prec1), to_python_float(batch_total))

        loss = loss*args.loss_scale
        
        # compute gradient and do SGD step
        if args.full_precision:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        else:
            model.zero_grad()
            loss.backward()
            model_grads_to_master_grads(model_params, master_params)
            for param in master_params:
                param.grad.data = param.grad.data/args.loss_scale
            optimizer.step()
            master_params_to_model_params(model_params, master_params)
            torch.cuda.synchronize()

        batch_time.update(time.time() - end)
        end = time.time()

        should_print = (batch_num%args.print_freq == 0) or (batch_num==trn_len)
        if should_print: log_batch(epoch, batch_num, trn_len, batch_time, losses, top1)
    return top1.avg, losses.avg

def validate(val_loader, model, criterion, epoch, start_time):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.eval()
    end = time.time()
    val_len = len(val_loader)

    for i,(input,target) in enumerate(val_loader):
        batch_num = i+1
        with torch.no_grad():
            output = model(input)
            loss = criterion(output, target).data
        batch_total = input.size(0)
        prec1 = accuracy(output.data, target)[0]
            
        losses.update(to_python_float(loss), batch_total)
        top1.update(to_python_float(prec1), batch_total)

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        should_print = (batch_num%args.print_freq == 0) or (batch_num==val_len)
        if should_print: log_batch(epoch, batch_num, val_len, batch_time, losses, top1)
            
    return top1.avg, losses.avg

def log_batch(epoch, batch_num, batch_len, batch_time, loss, top1):
    if args.local_rank==0 and args.verbose:
        output = ('Epoch: [{0}][{1}/{2}]\t' \
                + 'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t' \
                + 'Loss {loss.val:.4f} ({loss.avg:.4f})\t' \
                + 'Prec@1 {top1.val:.3f} ({top1.avg:.3f})').format(
                epoch, batch_num, batch_len, batch_time=batch_time, loss=loss, top1=top1)
        print(output)
        with open(f'{args.save_dir}/full.log', 'a') as f:
            f.write(output + '\n')
            
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = self.avg = self.sum = self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    corrrect_ks = correct(output, target, topk)
    batch_size = target.size(0)
    return [correct_k.float().mul_(100.0 / batch_size) for correct_k in corrrect_ks]

def correct(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).sum(0, keepdim=True)
        res.append(correct_k)
    return res


def sum_tensor(tensor):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.reduce_op.SUM)
    return rt

def reduce_tensor(tensor):
    rt = tensor.clone()
    dist.all_reduce(rt, op=dist.reduce_op.SUM)
    rt /= args.world_size
    return rt


In [9]:
# Filter out batch norm parameters and remove them from weight decay - gets us higher accuracy 93.2 -> 93.48
# https://arxiv.org/pdf/1807.11205.pdf
def bnwd_optim_params(model, model_params, master_params):
    bn_params, remaining_params = split_bn_params(model, model_params, master_params)
    return [{'params':bn_params,'weight_decay':0}, {'params':remaining_params}]


def split_bn_params(model, model_params, master_params):
    def get_bn_params(module):
        if isinstance(module, torch.nn.modules.batchnorm._BatchNorm): return module.parameters()
        accum = set()
        for child in module.children(): [accum.add(p) for p in get_bn_params(child)]
        return accum
    
    mod_bn_params = get_bn_params(model)
    zipped_params = list(zip(model_params, master_params))

    mas_bn_params = [p_mast for p_mod,p_mast in zipped_params if p_mod in mod_bn_params]
    mas_rem_params = [p_mast for p_mod,p_mast in zipped_params if p_mod not in mod_bn_params]
    return mas_bn_params, mas_rem_params
    

In [10]:
# Our own implementation of lars
class LARS(torch.optim.Optimizer):
    # SGD https://raw.githubusercontent.com/pytorch/pytorch/master/torch/optim/sgd.py
    # η (eta) = "trust" coefficient
    def __init__(self, params, lr, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False, eta=0.02, eps=1e-8, lars=True, local_mult=1):
        self.lr = lr
        self.lars = True
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov, eta=eta)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        self.local_mult = 1
        super().__init__(params, defaults)

    def __setstate__(self, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()
        if self.local_mult <= 0: self.lars = False
        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']
            eta = group['eta']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                if self.lars: local_lr = eta * torch.norm(p.data) / torch.norm(d_p)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.zeros_like(p.data)
                        buf.mul_(momentum).add_(d_p)
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf
                # print('Learning rate:', local_lr, local_lr*group['lr'])
                if self.lars: ratio = (1 - local_lr) * (1 - self.local_mult) + local_lr
                if self.lars: p.data.add_(-min(ratio*group['lr'], group['lr']), d_p)
                else: p.data.add_(-group['lr'], d_p)

        return loss

In [11]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter

In [28]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
args.full_precision = True
args.loss_scale = 1


# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

scale = 6
# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.02, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optimizer)
scheduler = Scheduler(optimizer, phases=[(1e-4*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])


sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=True, local_rank=0, loss_scale=1, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		33		0.2156		1.9826		2.3933		16.74
1		33		0.4323		1.4413		1.5176		47.87
2		33		0.6492		1.1644		1.5214		50.57
3		33		0.8688		0.9453		1.3104		56.48
4		33		1.0902		0.7748		0.9202		68.08
5		33		1.3072		0.668		0.7514		75.12
6		33		1.5251		0.5829		0.9944		68.46
7		33		1.7431		0.5426		1.2325		63.33
8		33		1.9636		0.5058		0.8011		72.93
9		33		2.1818		0.4776		1.1752		64.68
10		33		2.3996		0.4607		1.2616		60.52
11		33		2.6192		0.4648		0.9381		70.35
12		33		2.8375		0.4556		1.1569		6

In [14]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
args.full_precision = True
args.loss_scale = 1


# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

scale = 6
# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.02, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optimizer)
scheduler = Scheduler(optimizer, phases=[(1e-4*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])

sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))
    if epoch == 12: optimizer.lars = False

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=True, local_rank=0, loss_scale=1, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		33		0.2845		1.9879		2.6688		16.75
1		33		0.499		1.4477		1.668		41.04
2		33		0.7137		1.1732		1.5731		49.24
3		33		0.9286		0.9529		1.5037		57.04
4		33		1.1452		0.7878		1.0973		65.45
5		33		1.3622		0.6538		0.8876		71.79
6		33		1.5788		0.5832		1.7367		56.79
7		33		1.8001		0.5408		0.8762		72.98
8		33		2.0175		0.502		0.787		73.66
9		33		2.2344		0.4866		1.2111		62.56
10		33		2.4491		0.4803		1.0012		67.35
11		33		2.6658		0.4654		1.172		64.57
12		33		2.883		0.4601		1.1078		64.72


Process Process-275:
Process Process-273:
Process Process-276:
Process Process-278:
Process Process-277:
Process Process-274:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-279:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Process-280:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
T

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 1297, in pad
    if not np.asarray(pad_width).dtype.kind == 'i':


Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-805792b2015b>", line 43, in <module>
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
  File "<ipython-input-8-7d0ced679dc0>", line 21, in train
    for i,(input,target) in enumerate(trn_loader):
  File "<ipython-input-6-91dc9b208998>", line 66, in __iter__
    self.preload()
  File "<ipython-input-6-91dc9b208998>", line 54, in preload
    self.next_input, self.next_target = next(self.loaditer)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 280, in __next__
    idx, batch = self._get_batch()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 259, in _get_batch
    return self.

  File "<ipython-input-6-91dc9b208998>", line 2, in pad
    return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))


KeyboardInterrupt: 

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/datasets/cifar.py", line 121, in __getitem__
    img = self.transform(img)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/Image.py", line 2335, in frombytes
    im = new(mode, size)
KeyboardInterrupt
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/PIL/Image.py", line 2299, in new
    return Image()._new(core.fill(mode, size, color))
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 1301, in pad
    pad_width = _validate_lengths(narray, pad_width)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/paperspace/anaconda3/envs/fasta

In [13]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
# args.full_precision = True
# args.loss_scale = 1
scale = 6

# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = torch.optim.SGD(optim_params, lr=0, nesterov=True, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.01, momentum=args.momentum, weight_decay=args.weight_decay)
scheduler = Scheduler(optimizer, phases=[(1e-3*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])


sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=False, local_rank=0, loss_scale=512, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		33		0.1903		1.9546		1.7348		35.04
1		33		0.3061		1.6089		2.6223		32.54
2		33		0.4266		1.4725		1.5662		49.81
3		33		0.5437		1.0646		1.2858		56.45
4		33		0.6615		0.8873		1.8105		49.52
5		33		0.7811		0.7554		1.0411		65.34
6		33		0.8974		0.6541		0.9821		66.13
7		33		1.0139		0.5803		1.2272		58.83
8		33		1.1318		0.6226		0.8307		73.61
9		33		1.2491		0.5344		1.082		66.34
10		33		1.3671		0.4943		0.8002		73.18
11		33		1.4879		0.4836		2.1068		52.28
12		33		1.6045		0.9523		nan		1

Process Process-209:
Process Process-210:
Process Process-212:
Process Process-215:
Process Process-216:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Exception in thread Thread-30:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 71, in _worker_manager_loop
    r = in_queue.get()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/s

Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-ebfb043e484a>", line 43, in <module>
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
  File "<ipython-input-8-7d0ced679dc0>", line 61, in train
    torch.cuda.synchronize()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/cuda/__init__.py", line 339, in synchronize
    return torch._C._cuda_synchronize()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_trace

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-213:


KeyboardInterrupt: 

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-214:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 1301, in pad
    pad_width = _validate_lengths(narray, pad_width)
KeyboardInterrupt
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 1080, in _validate_lengths
    normshp = _normalize_shape(narray, number_elements)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/transforms/functional.py", line 83, in to_tensor
    return img.float().div(255)
KeyboardInterrupt
  File "/home/paperspace/anaconda3/envs/fastai/lib/

In [14]:
args.full_precision

False

In [15]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
# args.full_precision = True
# args.loss_scale = 1

scale = 4

# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = torch.optim.SGD(optim_params, lr=0, nesterov=True, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.01, momentum=args.momentum, weight_decay=args.weight_decay)
scheduler = Scheduler(optimizer, phases=[(1e-3*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])


sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=False, local_rank=0, loss_scale=512, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		49		0.1402		1.8387		1.4589		46.45
1		49		0.2551		1.4007		1.4751		50.81
2		49		0.3712		1.0809		1.6705		55.14
3		49		0.4873		0.879		1.2832		62.21
4		49		0.6022		0.6939		0.7004		75.96
5		49		0.7185		0.5765		0.7336		74.56
6		49		0.8339		0.5234		0.5688		80.19
7		49		0.9498		0.4719		0.7415		74.84
8		49		1.0675		0.4436		0.717		76.61
9		49		1.1844		0.4225		0.7966		72.42
10		49		1.3005		0.4355		1.0771		67.29
11		49		1.4161		0.4032		0.7548		74.76
12		49		1.5309		0.4309		0.66		7

In [16]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
args.full_precision = True
args.loss_scale = 1


# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

scale = 4
# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.02, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optimizer)
scheduler = Scheduler(optimizer, phases=[(1e-3*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])

sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))
    if epoch == 5: optimizer.lars = False
    if epoch == 10: break

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=True, local_rank=0, loss_scale=1, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		49		0.1249		1.9059		1.8281		36.99
1		49		0.2518		1.3144		1.2182		55.88
2		49		0.3788		1.0243		1.4156		54.51
3		49		0.5056		0.8188		1.0237		66.64
4		49		0.63		0.6801		1.008		65.66
5		49		0.7553		0.5885		0.6779		76.97
6		49		0.8643		nan		nan		0.31
7		49		0.9738		nan		nan		0.31
8		49		1.0832		nan		nan		0.31
9		49		1.1916		nan		nan		0.31
10		49		1.2996		nan		nan		0.31


In [16]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
args.full_precision = True
args.loss_scale = 1


# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

scale = 4
# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.02, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optimizer)
scheduler = Scheduler(optimizer, phases=[(1e-3*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])

sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

wean = 5.0
print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))
#     if epoch == 5: optimizer.lars = False
    optimizer.local_mult = max((wean - epoch)/wean, 0)
    print('Lars:', optimizer.lars)
    print('local mult:', optimizer.local_mult)
    if epoch == 10: break

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=True, local_rank=0, loss_scale=1, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		49		0.2205		1.931		1.6546		39.43
Lars: True
local mult: 1.0
1		49		0.4415		1.2995		1.183		56.39
Lars: True
local mult: 0.8
2		49		0.6603		1.232		1.1762		58.05
Lars: True
local mult: 0.6
3		49		0.8812		0.9543		1.1389		63.47
Lars: True
local mult: 0.4
4		49		1.1012		0.9376		0.8431		72.18
Lars: True
local mult: 0.2
5		49		1.3217		0.6632		0.9176		69.67
Lars: True
local mult: 0.0
6		49		1.5263		0.6041		0.5563		81.07
Lars: False
local mult: 0
7		49		1.7317		0.5003		0.7659		74.

In [13]:
if args.distributed:
    print('Distributed: initializing process group')
    torch.cuda.set_device(args.local_rank)
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url, world_size=args.world_size)
    assert(args.world_size == dist.get_world_size())
    print("Distributed: success (%d/%d)"%(args.local_rank, args.world_size))

model = ResNet18()
model = model.cuda()


if not args.full_precision: model = network_to_half(model)
if args.distributed: model = nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank)

# TESTING
args.full_precision = True
args.loss_scale = 1


# AS: todo: don't copy over weights as it seems to help accuracy
global model_params, master_params
if args.full_precision: model_params = master_params = model.parameters()
else: model_params, master_params = prep_param_lists(model)

# optim_params = bnwd_optim_params(model, model_params, master_params)
optim_params = master_params

scale = 6
# define loss function (criterion) and optimizer
criterion = F.cross_entropy
optimizer = LARS(optim_params, lr=0, nesterov=True, eta=0.02, momentum=args.momentum, weight_decay=args.weight_decay)
# optimizer = LARS(optimizer)
scheduler = Scheduler(optimizer, phases=[(1e-3*scale,2e-1*scale,15),(2e-1*scale,1e-2*scale,15),(1e-2*scale,0,5)])

sz = 32
trn_loader, val_loader = torch_loader(args.data, sz, 256*scale, 256*scale)

wean = 10.0
print(args)
print('\n\n')
print("epoch\t\tnum_batch\ttime (min)\ttrn_loss\tval_loss\taccuracy")
start_time = datetime.now() # Loading start to after everything is loaded
for epoch in range(scheduler.tot_epochs):
    trn_top1, trn_loss = train(trn_loader, model, criterion, optimizer, scheduler, epoch)
    val_top1, val_loss = validate(val_loader, model, criterion, epoch, start_time)

    time_diff = datetime.now()-start_time
    minutes = float(time_diff.total_seconds() / 60.0)
    # epoch   time   trn_loss   val_loss   accuracy     
    metrics = [str(round(i, 4)) for i in [epoch, len(trn_loader), minutes, trn_loss, val_loss, val_top1]]
    print('\t\t'.join(metrics))
#     if epoch == 5: optimizer.lars = False
    if epoch >= 15:
        optimizer.local_mult = max((21 - epoch)/(6), 0)
        print('Lars:', optimizer.lars)
        print('local mult:', optimizer.local_mult)

Files already downloaded and verified
Files already downloaded and verified
Namespace(batch_size=256, data='/home/paperspace/data/cifar10', dist_backend='nccl', dist_url='env://', distributed=False, full_precision=True, local_rank=0, loss_scale=1, momentum=0.9, phases='[(0,2e-1,15),(2e-1,1e-2,15),(1e-2,0,5)]', print_freq=200, save_dir=PosixPath('/home/paperspace/cluster/pytorch-cifar'), scale_lr=1, verbose=False, weight_decay=0.0005, workers=8, world_size=-1)



epoch		num_batch	time (min)	trn_loss	val_loss	accuracy
0		33		0.2847		1.9737		2.2577		20.57
1		33		0.5009		1.4178		1.49		49.22
2		33		0.7209		1.11		1.1421		60.39
3		33		0.9404		0.9082		1.3624		57.56
4		33		1.159		0.7457		0.8396		71.72
5		33		1.3783		0.6476		1.5352		57.22
6		33		1.5963		0.5787		0.7353		74.82
7		33		1.8164		0.5222		0.8139		73.37
8		33		2.0366		0.497		0.8971		70.06
9		33		2.2599		0.4817		0.7581		74.1
10		33		2.4815		0.461		1.1255		66.84
11		33		2.7003		0.4516		0.8122		72.06
12		33		2.9201		0.464		0.9269		69.47
13	